In [1]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.circuit import Parameter 



import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
theta_param, phi_param = Parameter('theta_param'), Parameter('phi_param') 

qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.ry(theta_param, qreg_q[0])
circuit.rz(phi_param, qreg_q[0])

circuit.rx(-np.pi/2, qreg_q[0])
circuit.sdg(qreg_q[1])
circuit.h(qreg_q[1])
circuit.tdg(qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.t(qreg_q[1])
circuit.h(qreg_q[1])
circuit.s(qreg_q[1])
circuit.cx(qreg_q[1], qreg_q[0])
circuit.rx(np.pi/2, qreg_q[0])
circuit.rx(np.pi/2, qreg_q[1])

circuit.rz(-phi_param, qreg_q[0])
circuit.ry(-theta_param, qreg_q[1])
circuit.measure(qreg_q[1], creg_c[1])

In [ ]:
N_shots=1024
provider = IBMQ.load_account()

## Santiago

In [7]:
# IBMQ.save_account(token=)


#backend = least_busy(provider.backends(simulator=False))
#print(backend)
backend = provider.backends.ibmq_santiago
id_file = 'eco_qcm_ids_santiago.txt'
result_txt='eco_results_ibm_santiago.txt'
data_sheet_file="eco_data_sheet_ibm_santiago.xlsx"

## Athens

In [ ]:
backend = provider.backends.ibmq_athens
id_file = 'eco_qcm_ids_athens.txt'
result_txt='eco_results_ibm_athens.txt'
data_sheet_file="eco_data_sheet_ibm_athens.xlsx"

## Vigo

In [ ]:
backend = provider.backends.ibmq_vigo
id_file = 'eco_qcm_ids_vigo.txt'
result_txt='eco_results_ibm_vigo.txt'
data_sheet_file="eco_data_sheet_ibm_vigo.xlsx"

## Ourense

In [ ]:
backend = provider.backends.ibmq_ourense
id_file = 'eco_qcm_ids_ourense.txt'
result_txt='eco_results_ibm_ourense.txt'
data_sheet_file="eco_data_sheet_ibm_ourense.xlsx"

In [8]:
num_pts = 1000
indices = np.arange(0, num_pts, dtype=float) + 0.5

theta = np.arccos(1 - 2*indices/num_pts)
phi = (np.pi * (1 + 5**0.5) * indices)%(2*np.pi)
coords=[]

for theta_elem,phi_elem in zip(theta,phi):
    coords.append((theta_elem,phi_elem))
    

In [ ]:
runningjobs=[]
job_ids=[]
index=0

while index*75 < num_pts:
    if((index+1)*75<num_pts):
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param : points[1]}) 
         for points in coords[index*75:(index+1)*75]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open(id_file, 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    else:
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open(id_file, 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    with open(id_file, 'a') as file:
        file.write(runningjobs[-1].job_id()+"\n")
    index=index+1

Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f2ce334e750>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f2ce3340310>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f2ce32f68d0>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f2ce1deeb10>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f2ce2efd810>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f2ce3128c50>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 0x7f2ce215d850>
Maximum number of jobs reached. Waiting...
The job finished with result <qiskit.result.result.Result object at 

In [ ]:
read_job_ids = open(id_file).read().splitlines()
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    results_probabilities.append((job["000"]/N_shots,job["001"]/N_shots))
results_probabilities=np.array(results_probabilities)

In [ ]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0","prob_1"])

In [ ]:
data_sheet.head()

In [ ]:
for (point,probabilities) in zip(target_points,results_probabilities):
    with open(result_txt, 'a') as file:
        file.write(str(point[0])+"\t"+str(point[1])+"\t"+str(probabilities[0])+"\t"+str(probabilities[1])+"\n")

In [ ]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0","prob_1"])

In [ ]:
data_sheet.head()

In [ ]:
# save to excel, optional
data_sheet.to_excel(data_sheet_file)

In [ ]:
data_sheet=pd.read_excel(data_sheet_file)